In [1202]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import phonenumbers 
import re

In [1203]:
#importing dataset
df=pd.read_csv('/Users/levy/Downloads/raw_customer_data.csv',sep=',')

In [1204]:
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', None)

In [1205]:
#renaming columns for better comprehension
df.columns = ['id', 'full_name', 'phone_number', 'mobile_number','country']

In [1206]:
# Normalizing non ASCII texts
# function to normalize non ascii
def normalizing_non_ascii(text):
    # Replace non-ASCII characters with their closest ASCII equivalent
    text = unidecode(text)
    return text
#appying/looping normalizing_non_ascii function through the full_name field
full_name_edited=[]
for rows in df['full_name']:
    rt = normalizing_non_ascii(rows)
    full_name_edited.append(rt)
df['full_name_edited'] = full_name_edited

In [1207]:
# identifying and defining single character replacements
replacements = {
   'j':'i', '@':'a',  'b':'o', 'y':'b', 'v':'e', 'z':'a', '!':'o', 'x':'s'
}
# defining function to replace irrelevant characters
def replace_characters_single(field):
    field=field.replace(replacements, regex=True, inplace=True) 
    return field
#appying replace_character function
df['full_name_edited'] = replace_characters_single(df['full_name_edited'])

In [1208]:
# identifying and defining double character replacements 
replacements = {
   'aa':'za','hb':'hy'}
# defining function to replace irrelevant characters
def replace_characters_double(field):
    field = field.replace(replacements, regex=True, inplace=True) 
    return field
#appying replace_character function
df['full_name_edited'] = replace_characters_double(df['full_name_edited'])

In [1209]:
# removing_number_from_text
def removing_number_from_text(dt):
    removing_numbers = lambda x: ''.join([i for i in x if not i.isdigit()])
    field = dt.apply(removing_numbers) 
    return field

df['full_name_edited'] = df['full_name_edited'].apply(removing_number_from_text)    

In [1210]:
df.dtypes

id                   object
full_name            object
phone_number         object
mobile_number       float64
country              object
full_name_edited     object
dtype: object

In [1211]:
# Capitalizing only the first letter of names
def capitalize_first_char(field):
    field=field.str.lower().str.title()
    return field
df['full_name_edited']=capitalize_first_char(df['full_name_edited'])


In [ ]:
# Define a function to validate phone numbers
def validate_phone(row):
    try:
        parsed =phonenumbers.parse(row['phone_number'], row['country'])
        if phonenumbers.is_possible_number(parsed) == True:
            # did not phonenumbers.PhoneNumberType.FIXED_LINE because fixed line and mobile tend to be mixed up 
            return pn.format_number(pn.parse(row['phone_number'], row['country']), pn.PhoneNumberFormat.NATIONAL)
        else:
            return None
    except phonenumbers.phonenumberutil.NumberParseException:
        return None

# Apply the function to the Phone and Country columns and create a new column for the results
df['phone_number_edited'] = df.apply(validate_phone, axis=1)


In [ ]:
# Define a function to validate mobile numbers
def validate_mobile(row):
    if row['mobile_number']!= np.nan:
        try:
            parsed = phonenumbers.parse(str(row['mobile_number']).split('.')[0], row['country'])
            if phonenumbers.is_possible_number(parsed) == True:
                return pn.format_number(pn.parse(str(row['mobile_number']).split('.')[0], row['country']), pn.PhoneNumberFormat.NATIONAL)
            else:
                return None
        #not applying phonenumbers.PhoneNumberType.MOBILE because it coud be mixed with fixed line during reg
        except phonenumbers.phonenumberutil.NumberParseException:
            return None
    else:
        return None
# Apply the validation function to the DataFrame
df['mobile_number_edited'] = df.apply(validate_mobile, axis=1)

# Print the updated DataFrame
df


In [1214]:
def remove_white_space(field):
    field=field.str.strip()
    return field
df['full_name_edited']=remove_white_space(df['full_name_edited'])
df['phone_number_edited']=remove_white_space(df['phone_number_edited'])
df['mobile_number_edited']=remove_white_space(df['mobile_number_edited'])

In [1215]:
df.duplicated().sum()

0

In [1216]:
df.dtypes

id                       object
full_name                object
phone_number             object
mobile_number           float64
country                  object
full_name_edited         object
phone_number_edited      object
mobile_number_edited     object
dtype: object

In [1217]:
df=df.drop(['full_name','phone_number','mobile_number'], axis=1)

In [1218]:
df.rename(columns={'full_name_edited':'full_name', 'phone_number_edited':'phone_number',\
              'mobile_number_edited':'mobile_number'},inplace=True)

In [1181]:
df.to_excel('/Users/levy/Documents/clarius_cx.xlsx',index=False)